In [1]:
import random

NUM_TESTS = 24
HEAP_SIZE = 1000
OBJECT_SIZE = 8
TEST_LENGTH = 10
SENTINEL_SIZE = 2 * 4 # size of start & end sentinel
MIN_BLOCK_SIZE = OBJECT_SIZE + 8;

# determine a random number of objects to allocate
def allocate(cur_heap_space, max_bytes):
    
    max_objects = max_bytes // OBJECT_SIZE
    return random.randint(1, max_objects)

def getNumAllocated(blocks):
    return sum(b[1] for b in blocks)

def getBiggestFreeBlock(blocks):
    max_size = 0
    for size, isAllocated in blocks:
        if not isAllocated and size > max_size:
            max_size = size
    return max_size

def getFreeBlock(blocks, num_objects): # index of first free block
    for i, (size, isAllocated) in enumerate(blocks):
        if not isAllocated and size >= (num_objects * OBJECT_SIZE):
            return i
    return -1 # No free block found

def get_nth_allocated_block(blocks, n): # one-indexed
    return next((block for block in blocks if block[1] and not (n := n-1)), None)

# takes one index
def get_nth_allocated_block_index(blocks, n):
    return ([i for i, block in enumerate(blocks) if block[1]][n-1] if n <= len([i for i, block in enumerate(blocks) if block[1]]) else -1)

# given the nth allocated block
def free_allocated_block(blocks, n):
    if n < 1 or n > len(blocks):
        raise ValueError("Invalid block index")
    if not blocks[n-1][1]:
        raise ValueError("Block is not allocated")

    # Free the block
    blocks[n-1] = (blocks[n-1][0], False)

    # Check if adjacent blocks are free and coalesce them
    coalesced = 0
    if n > 1 and not blocks[n-2][1]:
        size = blocks[n-2][0]
        blocks.pop(n-2)
        blocks[n-2] = (blocks[n-2][0] + size + 8, False)
        n -= 1
        coalesced += 1

    if n < len(blocks) and not blocks[n][1]:
        size = blocks[n][0]
        blocks.pop(n)
        blocks[n-1] = (blocks[n-1][0] + size + 8, False)
        coalesced += 1

    return blocks, coalesced


print(NUM_TESTS)
print()
for test_number in range(0, NUM_TESTS):
    cur_heap_space = HEAP_SIZE - SENTINEL_SIZE
    blocks = [((HEAP_SIZE - SENTINEL_SIZE), False)] # size, isAllocated
    
    for process_num in range(0, TEST_LENGTH):
        max_bytes = getBiggestFreeBlock(blocks)
        if getNumAllocated(blocks) == 0: # if no allocated blocks, we can't dealloc
            num_objects = allocate(cur_heap_space, max_bytes)
            index = getFreeBlock(blocks, num_objects)
            assert(index != -1)
            blocks[index] = (blocks[index][0] - OBJECT_SIZE * num_objects - SENTINEL_SIZE, False)
            blocks.insert(index, (OBJECT_SIZE * num_objects, True))
            cur_heap_space += -(OBJECT_SIZE * num_objects + SENTINEL_SIZE)
            print(num_objects)
        else:
            if (random.choice([True, False]) and max_bytes > MIN_BLOCK_SIZE): # allocate
                num_objects = allocate(cur_heap_space, max_bytes);
                index = getFreeBlock(blocks, num_objects)
                assert(index != -1)
                blocks[index] = (blocks[index][0] - OBJECT_SIZE * num_objects - SENTINEL_SIZE, False)
                blocks.insert(index, (OBJECT_SIZE * num_objects, True))
                cur_heap_space += -(OBJECT_SIZE * num_objects + SENTINEL_SIZE)
                print(num_objects)
            else: # deallocate
                dealloc_block_num = random.randint(1, getNumAllocated(blocks))
                dealloc_block_index = get_nth_allocated_block_index(blocks, dealloc_block_num)
                block_size = blocks[dealloc_block_index][0]
                new_block, coalesced = free_allocated_block(blocks, dealloc_block_index + 1)
                block = new_block
                cur_heap_space += block_size + (8 * coalesced)
                print(-dealloc_block_num)
    if (test_number < NUM_TESTS - 1): print()

24

90
19
9
3
-1
-2
-2
23
36
-3

33
-1
55
47
10
8
-2
-3
-2
46

100
7
-2
-1
38
-1
41
4
-1
-1

73
11
15
-2
9
2
3
-3
16
13

99
-1
120
3
-1
73
-1
-1
45
76

109
-1
113
-1
28
45
29
-3
-2
-1

41
-1
76
16
25
2
-2
-3
8
-1

63
36
7
3
9
-5
-2
-1
79
10

27
5
-1
-1
74
-1
66
39
6
-1

24
-1
111
7
4
-1
47
-1
39
24

32
-1
41
-1
62
47
3
-3
-2
29

120
-1
31
-1
54
4
30
-2
13
-1

43
57
21
-2
-1
-1
81
2
19
10

102
15
3
-1
72
-2
-1
-1
33
-1

123
-1
105
17
-1
-1
7
-1
40
-1

2
-1
96
10
-2
-1
63
-1
102
-1

26
69
-1
12
-1
-1
12
62
-1
-1

20
-1
105
18
-2
17
-2
5
12
-3

107
9
4
-1
38
42
16
-2
-4
-1

102
21
-1
67
6
3
-1
-2
-1
53

74
9
23
-1
37
-3
-1
28
-1
13

117
4
-1
-1
22
-1
18
-1
113
10

122
-1
66
20
-1
26
31
-2
-1
50

31
-1
103
10
-1
30
54
-1
16
8
